In [ ]:
# Import PySpark and other necessary libraries
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
!pip install pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, year, month
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

# Initialize Spark session
spark = SparkSession.builder.appName('Trade Flow Prediction').getOrCreate()


In [ ]:

# Load the datasets
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

big_df = spark.read.csv("/content/drive/My Drive/CIS 415/Project: Trade Flow/Trade_Flow_Big_Dataset.csv", header=True, inferSchema=True)
small_df = spark.read.csv('/content/drive/My Drive/CIS 415/Project: Trade Flow/Trade_Flow_Small_Dataset.csv', header=True, inferSchema=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Data Exploration and Cleaning
# Show the first few rows of the dataframe
small_df.show(5)

# Checking for missing values in each column
small_df.select([count(when(col(c).isNull(), c)).alias(c) for c in small_df.columns]).show()

# Removing rows with missing values
small_df = small_df.na.drop()


+----------+-----------------+----------------------+----------------+-----------+------------+
|      Date|Country_of_Origin|Country_of_Destination|Product_Category|Trade_Value|Trade_Volume|
+----------+-----------------+----------------------+----------------+-----------+------------+
|2016-09-24|            China|                 India| Pharmaceuticals|    1909.38|       166.0|
|2011-01-30|           Russia|                 China|        Textiles|    2343.78|       346.0|
|2012-07-02|            China|               Germany|        Textiles|   12596.81|       408.0|
|2018-06-23|           Brazil|                 India|       Machinery|     515.82|       207.0|
|2015-10-19|          Germany|                France|            Food|   21771.38|        54.0|
+----------+-----------------+----------------------+----------------+-----------+------------+
only showing top 5 rows

+----+-----------------+----------------------+----------------+-----------+------------+
|Date|Country_of_Orig

In [ ]:

# Feature Engineering
# Indexing categorical columns
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(small_df) for column in ['Country_of_Origin', 'Country_of_Destination', 'Product_Category']]

# Assemble vectors
assembler = VectorAssembler(inputCols=['Country_of_Origin_index', 'Country_of_Destination_index', 'Product_Category_index', 'Trade_Value'],
                            outputCol='features')


#Random Forest Model

# Define the model
rf = RandomForestRegressor(featuresCol='features', labelCol='Trade_Volume')

# Set up the pipeline
pipeline = Pipeline(stages=indexers + [assembler, rf])


In [ ]:

# Model Training
# Split the data into training and testing sets
train_data, test_data = small_df.randomSplit([0.8, 0.2], seed=42)

# Fit the model
rf_model = pipeline.fit(train_data)

# Make predictions
rf_predictions = rf_model.transform(test_data)

# Show sample predictions
rf_predictions.select('features', 'Trade_Volume', 'prediction').show(5)


+--------------------+------------+------------------+
|            features|Trade_Volume|        prediction|
+--------------------+------------+------------------+
|[6.0,1.0,4.0,1583...|       866.0|497.19927303692054|
|[3.0,4.0,1.0,591.97]|       275.0|460.02489015934964|
|[5.0,5.0,3.0,3009...|       241.0| 522.0876545479844|
|  [6.0,2.0,0.0,81.8]|       445.0| 483.9623281042682|
|[7.0,0.0,5.0,2289...|       388.0| 536.9819154618642|
+--------------------+------------+------------------+
only showing top 5 rows



In [ ]:

# Model Evaluation
evaluator = RegressionEvaluator(labelCol='Trade_Volume', predictionCol='prediction', metricName='rmse')
rmse = evaluator.evaluate(rf_predictions)
print(f'Root Mean Squared Error (RMSE) on test data = {rmse}')


Root Mean Squared Error (RMSE) on test data = 491.31511291415734


In [ ]:
# Linear Regression Model

from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='features', labelCol='Trade_Volume')

lr_pipeline = Pipeline(stages=indexers + [assembler, lr])

# Model Training
# Fit the model
lr_model = lr_pipeline.fit(train_data)

# Make predictions
lr_predictions = lr_model.transform(test_data)

# Show sample predictions
lr_predictions.select('features', 'Trade_Volume', 'prediction').show(5)

+--------------------+------------+------------------+
|            features|Trade_Volume|        prediction|
+--------------------+------------+------------------+
|[6.0,1.0,4.0,1583...|       866.0|503.11952540880725|
|[3.0,4.0,1.0,591.97]|       275.0| 498.1185476681084|
|[5.0,5.0,3.0,3009...|       241.0|505.20705766304013|
|  [6.0,2.0,0.0,81.8]|       445.0|496.87454394875215|
|[7.0,0.0,5.0,2289...|       388.0| 504.9385245570681|
+--------------------+------------+------------------+
only showing top 5 rows



In [ ]:
# Model Evaluation

lr_rmse = evaluator.evaluate(lr_predictions)
print(f'Root Mean Squared Error (RMSE) on test data = {rmse}')

Root Mean Squared Error (RMSE) on test data = 491.31511291415734


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, min, max, count, desc
from pyspark.sql import functions as F

# Initialize Spark session
spark = SparkSession.builder.appName('Descriptive Statistics').getOrCreate()

# Load the small dataset (ensure the file path is correct)
small_df = spark.read.csv('/content/drive/My Drive/CIS 415/Project: Trade Flow/Trade_Flow_Small_Dataset.csv', header=True, inferSchema=True)

# Show the first few rows of the DataFrame
small_df.show(5)

# Descriptive statistics for numerical columns
def descriptive_stats(df):
    numerical_cols = [col_name for col_name, dtype in df.dtypes if dtype in ['int', 'bigint', 'double']]
    stats_df = df.describe(numerical_cols)
    stats_df.show()

    for col_name in numerical_cols:
        df.select(
            mean(col_name).alias(f'Mean_{col_name}'),
            stddev(col_name).alias(f'Standard_Deviation_{col_name}'),
            min(col_name).alias(f'Min_{col_name}'),
            max(col_name).alias(f'Max_{col_name}')
        ).show()

# Frequency distribution for categorical columns
def frequency_distribution(df):
    categorical_cols = [col_name for col_name, dtype in df.dtypes if dtype == 'string']
    for col_name in categorical_cols:
        df.groupBy(col_name).count().orderBy(desc('count')).show()

# Call the functions to generate descriptive statistics
descriptive_stats(small_df)
frequency_distribution(small_df)

+----------+-----------------+----------------------+----------------+-----------+------------+
|      Date|Country_of_Origin|Country_of_Destination|Product_Category|Trade_Value|Trade_Volume|
+----------+-----------------+----------------------+----------------+-----------+------------+
|2016-09-24|            China|                 India| Pharmaceuticals|    1909.38|       166.0|
|2011-01-30|           Russia|                 China|        Textiles|    2343.78|       346.0|
|2012-07-02|            China|               Germany|        Textiles|   12596.81|       408.0|
|2018-06-23|           Brazil|                 India|       Machinery|     515.82|       207.0|
|2015-10-19|          Germany|                France|            Food|   21771.38|        54.0|
+----------+-----------------+----------------------+----------------+-----------+------------+
only showing top 5 rows

+-------+------------------+------------------+
|summary|       Trade_Value|      Trade_Volume|
+-------+------